In [1]:
import json
from collections import defaultdict

plik="przepisy.json"
with open(plik, 'r',encoding="utf-8") as f:
    przepisy= json.load(f)
    

"Czas przygotowania: 30 minut\nCzas kiszenia ogórków: 14 dni,
Liczba porcji: 4 x słoik 1 litr,
W 100 gramach ogórków:,
Wartość energetyczna,
20 kcal,
Węglowodany 4 g,
- W tym cukry 1 g
Białko 1 g,
Tłuszcze 0 g,
Dieta:,
bezglutenowa, wegańska, wegetariańska "

In [2]:



from openai import OpenAI
import os
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))




def setupingrideinets(recipy):

    ingredients=recipy
    prompt = f"""
    Przekształć poniższą listę składników na listę obiektów JSON, gdzie każdy składnik będzie miał następujące pola:

    - składnik: nazwa składnika (bez ilości i miary, np. "czosnek", "ogórki gruntowe", "sól kamienna")
    - ilość: liczba lub zapis słowny (np. "pół", "garść", "1,5")
    - miara: jednostka miary (np. "kg", "łyżki", "ząbki", "szklanka", "garść")
    - waga: waga w gramach, jeśli podana (np. "200 g", "50 g")

    Zwróć wynik jako listę obiektów JSON bez żadnych dodatkowych komentarzy ani tekstów.

    Składniki:
    {ingredients}
    """

    # Użycie właściwego modelu: "gpt-3.5-turbo"
    response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "system", "content": f"{prompt}"}
            
        ]
    )


    return response.choices[0].message.content


def spilt_values(line):
    if "x" in line:
        splitline=line.split("x")
        return splitline[0], splitline[1]
    else:
        if line[0]==" ":
            line = line[1:]
        splitline=line.split(" ")
        return splitline[0], splitline[1]


data = defaultdict(lambda: {"czas": {},"wartość energetyczna w 100g": {},"węglowodany": {},"cukier": {},"białko": {},"tłuszcze": {},"ingredient":[]})


for name in przepisy.keys():
    
    ingredient=przepisy[name]["ingredient"]
    data[name]["ingredient"]=setupingrideinets(ingredient)
    tempinfo=przepisy[name]["recipe_info"][0]
    tempinfo=tempinfo.split(",")
    print(tempinfo)
    for i in range(len(tempinfo) - 1):
        error=0
        info=tempinfo[i]
        if info.startswith("Czas"):
            info=info.split("\n")
            for i in info:
                infoarray=i.split(":")
                if len(infoarray)==2:
                    data[name]["czas"][infoarray[0]]=infoarray[1]
                else:
                    error=1
                
        elif info.startswith("Liczba porcji:"):
            porcje=info.split(":")
            data[name]["liczba porcji"]=porcje[1]

        elif info.startswith("Wartość energetyczna"):
            ilosc, wartosc =  spilt_values(tempinfo[i+1])
            data[name]["wartość energetyczna w 100g"]["ilosc"]=ilosc
            data[name]["wartość energetyczna w 100g"]["wartosc"]=wartosc

        elif info.startswith("Węglowodany"):
            zmienne=info.replace("Węglowodany ","")
            ilosc, wartosc =  spilt_values(zmienne)
            data[name]["węglowodany"]["ilosc"]=ilosc
            data[name]["węglowodany"]["wartosc"]=wartosc
        elif info.startswith("-"):
            zmienne=info.replace("- W tym cukry ","")
            ilosc, wartosc =  spilt_values(zmienne)
            data[name]["cukier"]["ilosc"]=ilosc
            data[name]["cukier"]["wartosc"]=wartosc
        elif info.startswith("Białko"):
            zmienne=info.replace("Białko ","")
            ilosc, wartosc =  spilt_values(zmienne)
            data[name]["białko"]["ilosc"]=ilosc
            data[name]["białko"]["wartosc"]=wartosc
        elif info.startswith("Tłuszcze"):
            zmienne=info.replace("Tłuszcze ","")
            ilosc, wartosc =  spilt_values(zmienne)
            data[name]["tłuszcze"]["ilosc"]=ilosc
            data[name]["tłuszcze"]["wartosc"]=wartosc
        elif info.startswith("Dieta"):
            data[name]["dieta"]=tempinfo[i+1].split(',')
        if error==1:
            del data[name]



with open("przepisy_idk.json", "w", encoding="utf-8") as f:
    json.dump(data, f, ensure_ascii=False, indent=2)



['Czas przygotowania: 5 minut\nCzas smażenia: 25 minut\nLiczba porcji: do 16 naleśników średnicy 24 cm', 'W 100 g:', 'Wartość energetyczna', '263 kcal', 'Węglowodany 32 g', '- W tym cukry 2 g', 'Białko 9 g', 'Tłuszcze 11 g', 'Dieta: wegetariańska ']
['Czas przygotowania: 10 minut\nCzas smażenia placków: 30 minut\nLiczba porcji: 14 grubszych placuszków', 'W 100 g:', 'Wartość energetyczna', '143 kcal', 'Węglowodany 14 g', '- W tym cukry 2 g', 'Białko 6 g', 'Tłuszcze 7 g', 'Dieta: wegetariańska ']
['Czas przygotowania: 30 minut\nCzas pasteryzacji: 6 minut\nCzas marynowania: 7 dni', 'Liczba porcji: 6 słoików po 900 ml', 'W 100 g:', 'Wartość energetyczna', '28 kcal', 'Węglowodany 6 g', '- W tym cukry 4 g', 'Białko 1 g', 'Tłuszcze 0 g', 'Dieta:', 'bezglutenowa', ' wegańska', ' wegetariańska ']
['Czas przygotowania: 30 minut\nCzas kiszenia ogórków: 14 dni', 'Liczba porcji: 4 x słoik 1 litr', 'W 100 gramach ogórków:', 'Wartość energetyczna', '20 kcal', 'Węglowodany 4 g', '- W tym cukry 1 g', '

    "ingredient": [
      "2 kg małych ogórków gruntowych",
      "2 małe główki czosnku",
      "duży pęczek kopru z baldachami",
      "kawałek korzenia chrzanu oraz liście - 10 cm",
      "2 litry wrzątku",
      "4 płaskie łyżki soli kamiennej",
      "można też dać: liście chrzanu, zielone liście dębu oraz liście czarnej porzeczki, winorośli lub wiśni"
    ],



Wartość | Miara | Składnik | Waga

In [7]:
def czy_liczba(wartosc):
    try:
        float(wartosc)  
        return True
    except ValueError:
        return False
jednostki_miary = [
    # Waga
    "g", "gram", "gramy", "gramów",
    "kg", "kilogram", "kilogramy", "kilogramów",
    "dag", "dekagram", "dekagramy", "dekagramów",
    # Objętość
    "ml", "mililitr", "mililitry", "mililitrów",
    "l", "litr", "litry", "litrów",
    "szklanka", "szklanki", "szklanek",
    "łyżka", "łyżki", "łyżek", "łyżka stołowa", "łyżki stołowe",
    "łyżeczka", "łyżeczki", "łyżeczek", "łyżeczka do herbaty", "łyżeczki do herbaty",
    # Sztuki / porcyjność
    "sztuka", "sztuki", "sztuk",
    "ząbek", "ząbki", "ząbków",
    "główka", "główki", "główek",
    "jajko", "jajka", "jajek",
    "pęczek", "pęczki", "pęczków",
    "gałązka", "gałązki", "gałązek",
    "liść", "liście", "listki", "liści",
    "kromka", "kromki", "kromek",
    "kostka", "kostki", "kostek",
    "opakowanie", "opakowania", "opakowań",
    "plaster", "plastry", "plastrów",
    # Potoczne kuchenne
    "szczypta", "szczypty", "szczypt",
    "garść", "garści",
    "łyczek", "łyczki", "łyczków",
    "kieliszek", "kieliszki", "kieliszków",
    # Nietypowe
    "cm", "centymetr", "centymetry", "centymetrów",
    "porcja", "porcje", "porcji"
]



def rozbij_skladnik(wiersz):
    tokens = wiersz.strip().split()
    ilosc = None
    miara = []
    skladnik = []
    
    i = 0
    # Szukamy liczby na początku
    if i < len(tokens) and (czy_liczba(tokens[i]) or re.match(r"^\d+/\d+$", tokens[i])):
        ilosc = tokens[i]
        i += 1

    # Szukamy miary (jednego lub dwóch tokenów)
    while i < len(tokens) and (tokens[i] in jednostki_miary or len(miara) < 2):
        miara.append(tokens[i])
        i += 1

    # Reszta to składnik
    skladnik = tokens[i:]

    return {
        "ilosc": ilosc,
        "miara": " ".join(miara) if miara else None,
        "skladnik": " ".join(skladnik) if skladnik else None
    }





import re
def get_jsonet(ingredients):

    data = defaultdict(lambda: { "miara": float,"value":str,"ing":str})
    for recipe_key in przepisy.keys():
        if recipe_key not in data:
            data[recipe_key] = {}
        ingredients= przepisy[recipe_key]["ingredient"]
        for ing in ingredients:
            tokens = ing.strip().split()
            ilosc = None
            miara = []
            skladnik = []
            
            i = 0
            # Szukamy liczby na początku
            if i < len(tokens) and (czy_liczba(tokens[i]) or re.match(r"^\d+/\d+$", tokens[i])):
                ilosc = tokens[i]
                i += 1

            # Szukamy miary (jednego lub dwóch tokenów)
            while i < len(tokens) and (tokens[i] in jednostki_miary or len(miara) < 2):
                miara.append(tokens[i])
                i += 1

            # Reszta to składnik
            skladnik = tokens[i:]
            print({
                "ilosc": ilosc,
                "miara": " ".join(miara) if miara else None,
                "skladnik": " ".join(skladnik) if skladnik else None
            })
            


            
    
                 
get_jsonet(plik)

{'ilosc': '2', 'miara': 'kg małych', 'skladnik': 'ogórków gruntowych'}
{'ilosc': '2', 'miara': 'małe główki', 'skladnik': 'czosnku'}
{'ilosc': None, 'miara': 'duży pęczek', 'skladnik': 'kopru z baldachami'}
{'ilosc': None, 'miara': 'kawałek korzenia', 'skladnik': 'chrzanu oraz liście - 10 cm'}
{'ilosc': '2', 'miara': 'litry wrzątku', 'skladnik': None}
{'ilosc': '4', 'miara': 'płaskie łyżki', 'skladnik': 'soli kamiennej'}
{'ilosc': None, 'miara': 'można też', 'skladnik': 'dać: liście chrzanu, zielone liście dębu oraz liście czarnej porzeczki, winorośli lub wiśni'}
{'ilosc': '2', 'miara': 'kg małych', 'skladnik': 'ogórków gruntowych'}
{'ilosc': '2', 'miara': 'szklanki octu', 'skladnik': 'spirytusowego 10 %'}
{'ilosc': '2', 'miara': 'litry wody', 'skladnik': '- 8 szklanek'}
{'ilosc': None, 'miara': 'około 3/4 szklanki', 'skladnik': 'cukru - 160 g'}
{'ilosc': '2', 'miara': 'pełne łyżki', 'skladnik': 'soli kamiennej - 50 g'}
{'ilosc': '2', 'miara': 'średnie marchewki', 'skladnik': '- 140 g'

['2 kg małych ogórków gruntowych', '2 małe główki czosnku', 'duży pęczek kopru z baldachami', 'kawałek korzenia chrzanu oraz liście - 10 cm', '2 litry wrzątku', '4 płaskie łyżki soli kamiennej', 'można też dać: liście chrzanu, zielone liście dębu oraz liście czarnej porzeczki, winorośli lub wiśni']
[
    {"składnik": "małe ogórki gruntowe", "ilość": "2", "miara": "kg", "waga": 2000},
    {"składnik": "małe główki czosnku", "ilość": "2", "miara": ""},
    {"składnik": "pęczek kopru z baldachami", "ilość": "duży", "miara": ""},
    {"składnik": "kawałek korzenia chrzanu oraz liście - 10 cm", "ilość": "", "miara": ""},
    {"składnik": "wrzątku", "ilość": "2", "miara": "litry"},
    {"składnik": "sol", "ilość": "4", "miara": "płaskie łyżki", "waga": 0},
    {"składnik": "liście chrzanu, zielone liście dębu oraz liście czarnej porzeczki, winorośli lub wiśni", "ilość": "", "miara": ""}
]
['2 kg małych ogórków gruntowych', '2 szklanki octu spirytusowego 10 %', '2 litry wody - 8 szklanek', 'o